In [1]:
import csv

In [5]:
table = []
with open('./ResTableMAL/resv_table.csv', 'r') as fh:
    csv_reader = csv.reader(fh)
    for row in csv_reader:
        print(row)
        table.append(row)

['x', '-', '-', '-', '-', 'x']
['-', 'x', '-', '-', '-', 'x']
['-', '-', 'x', '-', '-', '-']
['-', '-', 'x', '-', '-', 'x']
['-', '-', '-', '-', 'x', '-']


In [6]:
def get_forbidden_latencies(stage):
    lats = set()
    for i in range(len(stage)-1):
        if stage[i] != 'x':
            continue
        for j in range(i+1, len(stage)):
            if stage[j] == 'x':
                lats.add(j-i)
    return lats

In [7]:
forbid_lats = set()
for stage in table:
    forbid_lats |= get_forbidden_latencies(stage)
forbid_lats

{3, 4, 5}

In [8]:
max_forb = 0
coll_vec = 0
for lat in forbid_lats:
    coll_vec |= 1 << (lat-1)
    max_forb = max(max_forb, lat)
"{0:b}".format(coll_vec)

'11100'

In [9]:
from collections import defaultdict
graph = defaultdict(list)
done = set()
nodes = {coll_vec}
while True:
    for u in nodes:
        if u not in done:
            curr = u
            break
    else:
        break
    for i in range(max_forb+1):
        if not (curr >> i) & 1:
            state = (curr >> (i+1)) | coll_vec
            graph[curr].append((state, i+1))
            nodes.add(state)
    done.add(curr)

In [10]:
for u in graph:
    for v, w in graph[u]:
        print("{0:b} -> {1:b} : {2}".format(u, v, w))

11100 -> 11110 : 1
11100 -> 11111 : 2
11100 -> 11100 : 6
11110 -> 11111 : 1
11110 -> 11100 : 6
11111 -> 11100 : 6


In [11]:
["{0:b}".format(i) for i in nodes]

['11100', '11110', '11111']

In [12]:
def find_cycles(G, u, cycle):
    Cycles = []
    for v, w in G[u]:
        if v == cycle[0][0]:
            Cycles.append(cycle + [(v, w)])
        elif not any(v == c[0] for c in cycle):
            for c in find_cycles(G, v, cycle + [(v, w)]): Cycles.append(c)
    return Cycles

In [15]:
def is_rotated(c1, c2):
    c = c1 + c1
    for i in range(len(c)-len(c2)+1):
        if c[i:i+len(c2)] == c2: return True
    return False

In [16]:
greedy_cycle = None
MAL = float('inf')
Cycles = {}
print("Simple Cycles\tAvg. Latency")
for u in nodes:
    for cyc in find_cycles(graph, u, [(u, 0)]):
        wts = tuple(i[1] for i in cyc[1:])
        avg = sum(wts)/len(wts)
        if any((is_rotated(wts, c) and Cycles[c]==avg) for c in Cycles): continue
        Cycles[wts] = avg
        wts = '('+", ".join(map(str, wts))+')'
        if avg < MAL:
            MAL = avg
            greedy_cycle = wts
        print("{0:^13s}\t{1:^12.2f}".format(wts, avg))

Simple Cycles	Avg. Latency
  (1, 1, 6)  	    2.67    
   (1, 6)    	    3.50    
   (2, 6)    	    4.00    
     (6)     	    6.00    


In [17]:
print(f"Greedy Cycle : {greedy_cycle}\tMAL : {MAL:.2f}\nThroughput : {1/MAL:.2f}")

Greedy Cycle : (1, 1, 6)	MAL : 2.67
Throughput : 0.38
